#Importing Libraries

In [ ]:
import time
start_t = y
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
import fpg as fp
import timeit

In [ ]:
pip install memory_profiler

In [ ]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


#Reading Data


In [ ]:
#handle, text, lang = en, retweet_count, favourite_count
data = pd.read_csv("tweets.csv")
keep_columns = ["handle", "text", "lang"]

# removing columns of data which we won't be dealing with and keeping only keep_columns
remove_columns_list = list(set(data.columns)-set(keep_columns))
data.drop(columns = remove_columns_list, inplace = True)

# keeping rows which have tweets in english
data = data[data.lang == "en"]
data.drop(columns = ["lang"], inplace=True)
data.to_csv("twitter.csv")

hillary = data[data.handle == "HillaryClinton"]
hillary.drop(columns = ["handle"], inplace=True)
hillary.to_csv("Hillary.csv")

trump = data[data.handle == "realDonaldTrump"]
trump.drop(columns = ["handle"], inplace=True)
trump.to_csv("Trump.csv")

pd.set_option('display.max_columns', None)



/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


#Data Cleaning and Preprocessing

In [ ]:
# data cleaning
clean_data = ['',"will","a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "arent", "as", "at", "be", "because", "been", "before", "being","below", "between", "both", "but", "by","can", "cant", "cannot", "could", "couldnt", "did","didn't", "do", "does", "doesnt", "doing","dont", "down", "during", "each", "few", "for", "from", "further", "had", "hadnt","has", "hasnt", "have", "havent", "having","he", "hed", "hell", "hes", "her","here", "heres", "hers", "herself", "him", "himself", "his", "how", "how's", "i","id", "ill", "im", "ive", "if","in", "into", "is", "isnt", "it","its", "its", "itself", "lets", "me","more", "most", "mustnt", "my", "myself", "no", "nor", "not", "of", "off", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "shant", "she", "shed", "shell", "shes", "should", "shouldnt", "so", "some", "such", "than", "that", "thats", "the", "their", "theirs", "them","themselves", "then", "there", "theres", "these", "they", "theyd", "theyll", "theyre", "theyve", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was","wasnt", "we", "wed", "well", "were", "weve", "were", "werent", "what", "whats", "when", "when's", "where", "wheres", "which","while", "who", "whos", "whom", "why","whys", "with", "wont", "would", "wouldnt","you", "youd", "youll", "youre", "youve", "your", "yours","yourself", "yourselves"]
punctuation = list(string.punctuation + "”’â€™")

def clean(x):
    # make lowercase
    x = x.lower()

    #x = x.replace("â€™","") if x.find("â€™") else x
    #x = x.replace("â†","") if x.find("â€™") else x

    # to remove punctuation
    temp = ''
    for i in x:
        if i in punctuation or i.isnumeric():
            continue
        temp += i
    x = temp

    # remove links

    if x.find("http") != -1 :
        pos = x.find("http")
        space_pos = x.find(" ", pos)
        x = x[:pos] #x[0:pos] + x[space_pos:] if space_pos != -1 else x[0:pos]

    # removing common words

    words = x.split(" ")
    temp = ""
    for i in words:
        if i not in clean_data:
            temp += i+" "

    x = temp.strip()
    return x

def preprocessing(Data):
    transactions = []
    for i,j in Data.iterrows():
        str = j.text
        str = clean(str)
        Data.at[i,'text'] = str.split(" ")
        transactions.append(str.split(" "))
    Data.drop(Data.columns[[0]], axis = 1, inplace = True)
    Data.to_csv("trial.csv")
    return transactions

data_hillary = pd.read_csv("Hillary.csv")
# data = data.head(50) # to reduce number of rows while testing
transactions_h = preprocessing(data_hillary)

data_trump = pd.read_csv("Trump.csv")
# data = data.head(50) # to reduce number of rows while testing
transactions_t = preprocessing(data_trump)

#Training Dataset based on FP algorithm

In [ ]:
def support_count(x,Data):
    count = 0
    x = set(x)
    for j in Data['text']:
        j = set(j)
        if (x.issubset(j)):
            count += 1
    return count

def fpgrowth(transactions, minsup,Data):

    # generating frequent patterns
   
    patterns = fp.find_frequent_patterns(transactions, minsup)

    # generating association rules as {(left): ((right), confidence)}

    rules = fp.generate_association_rules(patterns, 0.7)
    
    
    rules_data = {}
    rules_data["Antecedent"] = list(rules.keys())
    rules_data["Consequent"] = [rules[i][0] for i in rules_data["Antecedent"]]
    rules_data["Confidence"] = [rules[i][1] for i in rules_data["Antecedent"]]

    # calculating support of RHS aka consequent

    support_RHS = []
    for i in rules_data["Consequent"]:
        a = support_count(i,Data)
        support_RHS.append(a*1.0/len(Data))
    # calculating support of consequent and antecedent
    support_total = []
    for i in range(len(rules_data["Confidence"])):
        a = support_count(rules_data["Antecedent"][i],Data)
        support_total.append(rules_data["Confidence"][i]*a/len(Data))    
    
    rules_data['RHS_support'] = support_RHS
    rules_data['Total_support'] = support_total

    # calculating support of LHS aka antecedent
    support_LHS = []
    for i in range(len(rules_data["Confidence"])):
        support_LHS.append(rules_data['Total_support'][i] / rules_data["Confidence"][i] )
    rules_data['LHS_support'] = support_LHS     

    # calculating lift

    lift = []
    for i in range(len(rules_data["Confidence"])):
        lift.append( rules_data["Confidence"][i] / rules_data['RHS_support'][i])
    rules_data['Lift'] = lift

    # storing results in .csv file

    results = pd.DataFrame.from_dict(rules_data)
    return results

#Executing Hillary Tweet analysis and sorting based on Lift

In [ ]:

results_Hillary = fpgrowth(transactions_h, 10, data_hillary) # 10 is the minimum support

results_Hillary.to_csv("Results - Hillary.csv") #adding sorted data

print("Results of FP Growth Algorithm for Hillary is saved in file Results - Hillary.csv")


Results of FP Growth Algorithm for Hillary is saved in file Results - Hillary.csv


In [ ]:
results_Hillary.nlargest(n = 20, columns = 'Lift')


,Antecedent,Consequent,Confidence,RHS_support,Total_support,LHS_support,Lift
15,"(minimum,)","(wage,)",1.125000,0.005775,0.005414,0.004812,194.812500
4,"(human,)","(rights,)",2.583333,0.013474,0.006630,0.002567,191.720238
14,"(pay, share)","(fair,)",0.909091,0.004812,0.003208,0.003529,188.909091
16,"(wage,)","(minimum,)",0.857143,0.004812,0.004950,0.005775,178.114286
12,"(fair, pay)","(share,)",1.000000,0.007700,0.003208,0.003208,129.875000
33,"(college, debt)","(student,)",0.909091,0.007700,0.003208,0.003529,118.068182
11,"(star,)","(gold,)",0.866667,0.007379,0.004171,0.004812,117.452174
29,"(mike,)","(pence,)",0.850000,0.007379,0.005454,0.006416,115.193478
5,"(oval,)","(office,)",0.833333,0.007379,0.003208,0.003850,112.934783
30,"(pence,)","(mike,)",0.708333,0.006416,0.005227,0.007379,110.393750


#Executing Trump Tweet analyses and sorting based on Lift

In [ ]:

%memit results_Trump = fpgrowth(transactions_t, 10, data_trump) # 10 is the minimum suppor

results_Trump.to_csv("Results - Trump.csv")
print("Results of FP Growth Algorithm for Trump is saved in file Results - Trump.csv")


peak memory: 229.90 MiB, increment: 0.01 MiB
Results of FP Growth Algorithm for Trump is saved in file Results - Trump.csv


In [ ]:
results_Trump.nlargest(n = 20, columns = 'Lift')



,Antecedent,Consequent,Confidence,RHS_support,Total_support,LHS_support,Lift
13,"(elizabeth, native)","(american, goofy)",1.100000,0.003513,0.003513,0.003194,313.100000
9,"(core,)","(common,)",1.100000,0.003833,0.003162,0.002874,287.008333
8,"(common,)","(core,)",0.785714,0.002874,0.003011,0.003833,273.341270
2,"(graham,)","(lindsey,)",0.769231,0.002874,0.002948,0.003833,267.606838
3,"(lindsey,)","(graham,)",1.000000,0.003833,0.002874,0.002874,260.916667
0,"(prayers,)","(thoughts,)",0.769231,0.003194,0.003194,0.004152,240.846154
1,"(thoughts,)","(prayers,)",1.000000,0.004152,0.003194,0.003194,240.846154
30,"(born, ted)","(canada,)",1.100000,0.004791,0.002811,0.002555,229.606667
35,"(hillary, wall)","(street,)",1.166667,0.005110,0.004099,0.003513,228.302083
27,"(law, officers)","(enforcement,)",1.000000,0.004791,0.003513,0.003513,208.733333


In [ ]:
end_t = time.time()
print(start_t)
print(end_t)
print(end_t - start_t)

1603342582.9658668
1603342589.5387442
6.572877407073975
